# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,-9.47,79,68,1.94,GS,1723110289
1,1,stanley,54.8680,-1.6985,17.07,74,40,4.12,GB,1723110290
2,2,cambria,35.5641,-121.0808,17.83,85,26,1.08,US,1723110291
3,3,port-aux-francais,-49.3500,70.2167,0.78,78,100,10.15,TF,1723110292
4,4,khairpur nathan shah,27.0936,67.7337,39.13,41,12,2.09,PK,1723110293


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot 
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',  
    geo=True,      
    tiles='OSM',   
    color='City',  
    size=5,   
    hover_cols=['City', 'Humidity', 'Max Temp', 'Cloudiness', 'Wind Speed'],  # Columns to display on hover
    title='City Weather Data'
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Max Temp,Cloudiness,Wind Speed)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Define ideal weather conditions
temp_low = 21
temp_high = 27
max_wind_speed = 4.5
max_cloudiness = 0

# Filter DataFrame for ideal weather conditions
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= temp_low) & 
    (city_data_df['Max Temp'] <= temp_high) &
    (city_data_df['Wind Speed'] < max_wind_speed) &
    (city_data_df['Cloudiness'] <= max_cloudiness)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df

# Display sample data
ideal_weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,colonia,50.9333,6.9500,21.78,61,0,3.09,DE,1723110178
36,36,vila velha,-20.3297,-40.2925,21.64,84,0,3.13,BR,1723110330
105,105,east london,-33.0153,27.9116,22.52,33,0,4.12,ZA,1723110321
117,117,newman,37.3138,-121.0208,21.71,64,0,2.13,US,1723110418
119,119,lethem,3.3803,-59.7968,23.01,92,0,2.29,GY,1723110420
175,175,barra,-11.0894,-43.1417,22.06,64,0,2.08,BR,1723110483
185,185,lexington,37.9887,-84.4777,21.19,83,0,3.74,US,1723110455
309,309,kailua-kona,19.6406,-155.9956,24.04,86,0,2.57,US,1723110631
335,335,astara,38.5000,48.6667,25.99,50,0,4.44,AZ,1723110659
337,337,ceres,37.5949,-120.9577,22.47,49,0,3.00,US,1723110661


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Step 3: Create a new DataFrame called `hotel_df`

# Use the Pandas copy function to create a DataFrame called hotel_df
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
15,colonia,DE,50.9333,6.9500,61,
36,vila velha,BR,-20.3297,-40.2925,84,
105,east london,ZA,-33.0153,27.9116,33,
117,newman,US,37.3138,-121.0208,64,
119,lethem,GY,3.3803,-59.7968,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel


# Define the radius of the search area in meters
radius = 10000

# Set your Geoapify API key here
api_key = geoapify_key

# Set the base URL for the Geoapify Places API
base_url = "https://api.geoapify.com/v2/places"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Set parameters for the API request
    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{lng},{lat},{radius}",
        "bias": f"proximity:{lng},{lat}",
        "apiKey": api_key
    }

    # Make the API request
    response = requests.get(base_url, params=params)
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
colonia - nearest hotel: Wasserturm Hotel Cologne
vila velha - nearest hotel: Hotel Prainha
east london - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
lethem - nearest hotel: Hotel Amazonas
barra - nearest hotel: No hotel found
lexington - nearest hotel: Extended Stay America
kailua-kona - nearest hotel: Kona Seaside Hotel
astara - nearest hotel: No hotel found
ceres - nearest hotel: Howard Johnson by Wyndham Modesto Ceres
kolonia - nearest hotel: Wasserturm Hotel Cologne
breves - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
longview - nearest hotel: Texas Lodge
saint-francois - nearest hotel: Chez Lily
darwin - nearest hotel: Mantra Pantanas Darwin
yelizovo - nearest hotel: Николина Усадьба
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
15,colonia,DE,50.9333,6.9500,61,Wasserturm Hotel Cologne
36,vila velha,BR,-20.3297,-40.2925,84,Hotel Prainha
105,east london,ZA,-33.0153,27.9116,33,No hotel found
117,newman,US,37.3138,-121.0208,64,No hotel found
119,lethem,GY,3.3803,-59.7968,92,Hotel Amazonas
175,barra,BR,-11.0894,-43.1417,64,No hotel found
185,lexington,US,37.9887,-84.4777,83,Extended Stay America
309,kailua-kona,US,19.6406,-155.9956,86,Kona Seaside Hotel
335,astara,AZ,38.5000,48.6667,50,No hotel found
337,ceres,US,37.5949,-120.9577,49,Howard Johnson by Wyndham Modesto Ceres


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',  
    geo=True,     
    tiles='OSM',   
    color='Humidity', 
    size=5,   
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'], 
    title='City Weather Data with Nearby Hotels'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)